# Этапы 3 (Онтология Brawl Stars)


In [1]:
# !pip install rdflib SPARQLWrapper -q

In [2]:
from rdflib import Graph, Namespace, Literal
from rdflib.namespace import RDF, RDFS, XSD
from pathlib import Path
import csv

## 1. Загрузка онтологии

Парсим онтологию, которую предварительно сохранили из Protégé в формате Turtle (`.ttl`).  


In [3]:
ONTOLOGY_PATH = Path("brawl_ontology.ttl")

g = Graph()
g.parse(str(ONTOLOGY_PATH), format="turtle")

print("Триплов в графе до добавления данных:", len(g))

BS = Namespace("http://example.org/brawlstars#")
g.bind("bs", BS)

print("Несколько бойцов:")
for s, p, o in list(g.triples((None, RDF.type, BS.Brawler)))[:10]:
    label = g.value(s, RDFS.label)
    print("-", s, "=>", label)

Триплов в графе до добавления данных: 876
Несколько бойцов:
- http://example.org/brawlstars#Shelly => Shelly
- http://example.org/brawlstars#Colt => Colt
- http://example.org/brawlstars#Nita => Nita
- http://example.org/brawlstars#ElPrimo => El Primo
- http://example.org/brawlstars#Poco => Poco
- http://example.org/brawlstars#Jessie => Jessie
- http://example.org/brawlstars#Brock => Brock
- http://example.org/brawlstars#Dynamike => Dynamike
- http://example.org/brawlstars#Bo => Bo
- http://example.org/brawlstars#Tick => Tick


## 2. Внешний источник данных

В качестве внешнего источника данных используется CSV-файл `brawlers.csv` со структурой:

- `brawler_id` — идентификатор бойца
- `brawler_label` — человекочитаемое имя
- `rarity` — редкость
- `class` — класс
- `health` — базовое здоровье
- `movementSpeed` — скорость
- `suitedModes` — список подходящих режимов
- `counters` — список бойцов, которых данный боец контрит


In [4]:
CSV_PATH = Path("brawlers.csv")

## 3. Загрузка данных из CSV и добавление в онтологию


In [5]:
count_before = len(g)

with CSV_PATH.open(encoding="utf-8") as f:
    reader = csv.DictReader(f)
    for row in reader:
        b_id = row["brawler_id"].strip()
        if not b_id:
            continue

        # URI бойца
        b = BS[b_id]

        # Тип и лейбл
        g.add((b, RDF.type, BS.Brawler))
        g.add((b, RDFS.label, Literal(row["brawler_label"].strip(), lang="en")))

        # Редкость
        rarity = row.get("rarity", "").strip()
        if rarity:
            rarity_uri = getattr(BS, rarity)
            g.add((b, BS.hasRarity, rarity_uri))

        # Класс
        cls = row.get("class", "").strip()
        if cls:
            class_uri = getattr(BS, cls)
            g.add((b, BS.hasClass, class_uri))

        # Здоровье
        health = row.get("health", "").strip()
        if health:
            try:
                g.add((b, BS.hasHealth, Literal(int(health), datatype=XSD.integer)))
            except ValueError:
                print(f"Предупреждение: не удалось преобразовать здоровье '{health}' для бойца {b_id}")

        # Скорость
        speed = row.get("movementSpeed", "").strip()
        if speed:
            g.add((b, BS.movementSpeed, Literal(speed)))

        # Подходящие режимы
        modes_raw = row.get("suitedModes", "")
        modes = [m.strip() for m in modes_raw.split(";") if m.strip()]
        for m in modes:
            mode_uri = getattr(BS, m)
            g.add((b, BS.suitedForMode, mode_uri))

        # Кого он контрит
        counters_raw = row.get("counters", "")
        counters = [c.strip() for c in counters_raw.split(";") if c.strip()]
        for c in counters:
            counter_uri = getattr(BS, c)
            g.add((b, BS.counters, counter_uri))

count_after = len(g)
print("Триплов было: ", count_before)
print("Триплов стало:", count_after)
print("Добавлено триплов:", count_after - count_before)

Триплов было:  876
Триплов стало: 1244
Добавлено триплов: 368


## 4. Примеры SPARQL-запросов по обновлённой онтологии

Сделаем селект-запрос, который выведет бойцов, для которых в онтологии присутствуют:

- `bs:hasHealth`,
- `bs:movementSpeed`,
- хотя бы один `bs:suitedForMode`.


In [6]:
# Пример 1: Простой SELECT запрос
query_simple = """
PREFIX bs: <http://example.org/brawlstars#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?b ?label ?health ?speed ?modeLabel
WHERE {
  ?b a bs:Brawler ;
     rdfs:label ?label ;
     bs:hasHealth ?health ;
     bs:movementSpeed ?speed ;
     bs:suitedForMode ?mode .
  ?mode rdfs:label ?modeLabel .
}
ORDER BY ?label ?modeLabel
LIMIT 10
"""

print("--- Простой список бойцов и режимов ---")
for row in g.query(query_simple):
    print(f"{row.label} — HP={row.health}, speed={row.speed}, mode={row.modeLabel}")

--- Простой список бойцов и режимов ---
8-Bit — HP=3300, speed=Normal, mode=Bounty
8-Bit — HP=3300, speed=Normal, mode=Duels
8-Bit — HP=3300, speed=Normal, mode=Gem Grab
8-Bit — HP=3300, speed=Normal, mode=Hot Zone
8-Bit — HP=3300, speed=Normal, mode=Knockout
Amber — HP=4600, speed=Normal, mode=Duo Showdown
Amber — HP=4600, speed=Normal, mode=Heist
Amber — HP=4600, speed=Normal, mode=Hot Zone
Amber — HP=4600, speed=Normal, mode=Knockout
Amber — HP=4600, speed=Normal, mode=Wipeout


In [7]:
# Пример 2: "Самые универсальные бойцы" (Агрегация и фильтрация)
# Задача: Найти бойцов, которые подходят для >= 3 режимов.
# Использует: GROUP BY, COUNT, HAVING, ORDER BY DESC

query_versatile = """
PREFIX bs: <http://example.org/brawlstars#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?brawlerName (COUNT(?mode) AS ?modeCount) ?rarityLabel
WHERE {
  ?b a bs:Brawler ;
     rdfs:label ?brawlerName ;
     bs:hasRarity ?rarity ;
     bs:suitedForMode ?mode .

  ?rarity rdfs:label ?rarityLabel .
}
GROUP BY ?brawlerName ?rarityLabel
HAVING (COUNT(?mode) >= 3)
ORDER BY DESC(?modeCount) ASC(?brawlerName)
"""

print("\n--- Топ универсальных бойцов (3+ режима) ---")
print(f"{'Боец':<15} | {'Режимов':<8} | {'Редкость'}")
print("-" * 40)

for row in g.query(query_versatile):
    print(f"{row.brawlerName:<15} | {row.modeCount:<8} | {row.rarityLabel}")


--- Топ универсальных бойцов (3+ режима) ---
Боец            | Режимов  | Редкость
----------------------------------------
Shelly          | 7        | Starting
Brock           | 6        | Super Rare
El Primo        | 6        | Rare
8-Bit           | 5        | Super Rare
Amber           | 5        | Legendary
Bonnie          | 5        | Epic
Colt            | 5        | Rare
Crow            | 5        | Legendary
Griff           | 5        | Epic
Poco            | 5        | Rare
Sprout          | 5        | Mythic
Tara            | 5        | Mythic
Tick            | 5        | Super Rare
Barley          | 4        | Rare
Bea             | 4        | Epic
Bibi            | 4        | Epic
Bo              | 4        | Super Rare
Brutus          | 4        | Legendary
Bull            | 4        | Rare
Byron           | 4        | Mythic
Carl            | 4        | Super Rare
Darryl          | 4        | Super Rare
Dynamike        | 4        | Super Rare
Edgar           | 4       

In [8]:
# Пример 3: "Охотники на Танков" (Сложная связь)
# Задача: Найти, кто контрит бойцов класса "Tank", и вывести здоровье "жертвы".
# Использует: Связь через промежуточное свойство (counters -> hasClass)

query_tank_killers = """
PREFIX bs: <http://example.org/brawlstars#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?hunterName ?preyName ?preyHP
WHERE {
  # Охотник (кто контрит)
  ?hunter a bs:Brawler ;
          rdfs:label ?hunterName ;
          bs:counters ?prey .

  # Жертва (кого контрят) должна быть Танком
  ?prey a bs:Brawler ;
        rdfs:label ?preyName ;
        bs:hasHealth ?preyHP ;
        bs:hasClass bs:Tank .
}
ORDER BY DESC(?preyHP)
"""

print("\n--- Кто контрит Танков? ---")
for row in g.query(query_tank_killers):
    print(f"{row.hunterName} контрит {row.preyName}")


--- Кто контрит Танков? ---
Colt контрит Brutus
Poco контрит Brutus
Grom контрит Brutus
Bonnie контрит Brutus
Byron контрит Brutus
Poco контрит Ash
Dynamike контрит Ash
Rico контрит Ash
Carl контрит Ash
Bea контрит Ash
Barley контрит Ash
Colonel Ruffs контрит Ash
Gus контрит Ash
Brock контрит Frank
Carl контрит Frank
Piper контрит Frank
Max контрит Frank
Colonel Ruffs контрит Frank
Tick контрит Rosa
Rico контрит Rosa
Bea контрит Rosa
Bonnie контрит Rosa
Sandy контрит Rosa
Nova контрит Rosa
Colt контрит El Primo
Penny контрит El Primo
Piper контрит El Primo
Max контрит El Primo
Spike контрит El Primo
Janet контрит El Primo
Gus контрит El Primo
8-Bit контрит Bull
Piper контрит Bull
Pam контрит Bull
Nani контрит Bull
Grom контрит Bull
Spike контрит Bull
Barley контрит Bull
Sandy контрит Bull
Nova контрит Bull
Brock контрит Darryl
Dynamike контрит Darryl
Tick контрит Darryl
8-Bit контрит Darryl
Penny контрит Darryl
Piper контрит Darryl
Pam контрит Darryl
Nani контрит Darryl
Byron контрит 

In [9]:
# Пример 4: Синергия в действии
# Задача: Найти пары бойцов, которые синергируют друг с другом и подходят для одного и того же режима.
# Использует: JOIN по свойству bs:suitedForMode для двух разных субъектов.

query_synergy_modes = """
PREFIX bs: <http://example.org/brawlstars#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?b1Name ?b2Name ?modeName
WHERE {
  # Боец 1 синергирует с Бойцом 2
  ?b1 bs:synergizesWith ?b2 ;
      rdfs:label ?b1Name ;
      bs:suitedForMode ?mode .

  # Боец 2 тоже должен подходить для этого режима
  ?b2 rdfs:label ?b2Name ;
      bs:suitedForMode ?mode .

  ?mode rdfs:label ?modeName .
}
ORDER BY ?modeName
"""

print("\n--- Синергирующие пары и их общие режимы ---")
for row in g.query(query_synergy_modes):
    print(f"[{row.modeName}] Пара: {row.b1Name} + {row.b2Name}")


--- Синергирующие пары и их общие режимы ---
[Brawl Ball] Пара: Poco + El Primo
[Brawl Ball] Пара: Poco + Frank
[Brawl Ball] Пара: Poco + Bibi
[Duels] Пара: Poco + Bibi
[Gem Grab] Пара: Poco + El Primo
[Gem Grab] Пара: Poco + Frank
[Hot Zone] Пара: Poco + El Primo
[Hot Zone] Пара: Poco + Frank
[Knockout] Пара: Poco + El Primo
[Knockout] Пара: Poco + Bibi


In [10]:
# Пример 5: Поиск разрушителей стен (Regex)
# Задача: Найти бойцов, чей Супер в описании содержит слова о разрушении стен.
# Использует: FILTER REGEX, поиск по строкам.

query_wall_breakers = """
PREFIX bs: <http://example.org/brawlstars#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?bName ?superName ?desc
WHERE {
  ?b a bs:Brawler ;
     rdfs:label ?bName ;
     bs:hasSuper ?super .
     
  ?super rdfs:label ?superName ;
         bs:description ?desc .
         
  # Ищем корни слов "стена", "ломать", "разрушать", "препятствия" (case-insensitive)
  FILTER REGEX(?desc, "(стен|лом|разруш|препятств)", "i")
}
"""

print("\n--- Разрушители ландшафта (по описанию Супера) ---")
for row in g.query(query_wall_breakers):
    # Обрезаем описание для краткости
    short_desc = (row.desc[:50] + '..') if len(row.desc) > 50 else row.desc
    print(f"{row.bName} ({row.superName}): {short_desc}")


--- Разрушители ландшафта (по описанию Супера) ---
Shelly (Super Shell): Конусный залп, ломающий стены и отбрасывающий враг..
Colt (Bullet Storm): Длинная линия снарядов, пробивающих препятствия.
El Primo (Flying Elbow Drop): Прыжок с приземлением, разрушающий стены.
Dynamike (Big Barrel O' Boom): Большой урон по зоне, ломает стены.
Darryl (Barrel Roll): Катится, отскакивая от стен.
Frank (Stunning Blow): Оглушение в конусе, ломает стены.
Bibi (Spitball): Пускает шар, отскакивающий от стен.
Grom (Foot Patrol): Большая бомба, ломающая укрытия.
Sprout (Garden Mulcher): Создает растительную стену.


In [15]:
# Пример 6: Цепочки контр-пиков (A -> B -> C)
# Задача: Найти цепочки из трех бойцов, где первый контрит второго, а второй — третьего.
# Использует: Двойной проход по свойству bs:counters.

query_counter_chain = """
PREFIX bs: <http://example.org/brawlstars#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?killerName ?victimName ?victimVictimName
WHERE {
  # Первое звено
  ?killer bs:counters ?victim ;
          rdfs:label ?killerName .
          
  # Второе звено
  ?victim bs:counters ?lastVictim ;
          rdfs:label ?victimName .
          
  ?lastVictim rdfs:label ?victimVictimName .
  
  # Исключаем тривиальные циклы (A -> B -> A), если нужно
  FILTER(?killer != ?lastVictim) 
}
LIMIT 10
"""

print("\n--- Цепочки контр-пиков (X > Y > Z) ---")
for row in g.query(query_counter_chain):
    print(f"{row.killerName} бьёт {row.victimName}, который бьёт {row.victimVictimName}")


--- Цепочки контр-пиков (X > Y > Z) ---
Piper бьёт El Primo, который бьёт Brock
Piper бьёт El Primo, который бьёт Edgar
Piper бьёт El Primo, который бьёт Leon
Colt бьёт El Primo, который бьёт Piper
Colt бьёт El Primo, который бьёт Brock
Colt бьёт El Primo, который бьёт Edgar
Colt бьёт El Primo, который бьёт Leon
Janet бьёт El Primo, который бьёт Piper
Janet бьёт El Primo, который бьёт Brock
Janet бьёт El Primo, который бьёт Edgar


In [14]:
# Пример 7: Мета режима Brawl Ball
# Задача: Посчитать, сколько бойцов каждого класса подходит для режима Brawl Ball.
# Использует: Фильтрация по конкретному URI режима, GROUP BY.

query_brawlball_meta = """
PREFIX bs: <http://example.org/brawlstars#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>

SELECT ?classLabel (COUNT(?b) as ?count)
WHERE {
  ?b a bs:Brawler ;
     bs:hasClass ?cls ;
     bs:suitedForMode bs:BrawlBall . # Конкретный URI режима
     
  ?cls rdfs:label ?classLabel .
}
GROUP BY ?classLabel
ORDER BY DESC(?count)
"""

print("\n--- Распределение классов в Brawl Ball ---")
for row in g.query(query_brawlball_meta):
    print(f"{row.classLabel}: {row['count']}")


--- Распределение классов в Brawl Ball ---
Controller: 6
Support: 6
Damage Dealer: 4
Tank: 4
Marksman: 1
Assassin: 1


## 5. Сохранение обновлённой онтологии

In [13]:
OUTPUT_PATH = Path("brawl_ontology_enriched.ttl")
g.serialize(destination=str(OUTPUT_PATH), format="turtle")

<Graph identifier=Nb32c2e878f484442bd1d3a9fd62aa718 (<class 'rdflib.graph.Graph'>)>